In [203]:
"""
Getting the category and attributes from the excel file and matching it with the data from the json file.
"""
import json
import pandas as pd
from fuzzywuzzy import fuzz

def post_process_attribute_df(excel_file_path):
    df = pd.read_csv(excel_file_path)
    df = df.transpose()
    df.head()
    df = df.drop("ID")
    df.columns = df.iloc[0]
    df = df.drop(["ATTRIBUTE"])
    df.reset_index(drop=True, inplace=True)
    df.fillna("empty", inplace = True)
    return df


with open('test.json', encoding='utf-8') as f:
    data = json.load(f)

def df_to_dict(df):
    category_dict = dict()
    df_dict = df.to_dict()
    for i in df_dict:
        category_dict[i] = list(df_dict[i].values())
    return category_dict




    
def post_process_category_dict(category_dict):
    for i in category_dict:
        category_dict[i] =  [x for x in category_dict[i] if x != 'empty']
    return category_dict


excel_file_path = 'NER_Attributes.csv'
df=post_process_attribute_df(excel_file_path)

category_dict = df_to_dict(df)
category_dict = post_process_category_dict(category_dict)

/tmp/ipykernel_23144/358178464.py:25: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_dict = df.to_dict()


In [215]:
# import necessary libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
import copy


# download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english') + list(string.punctuation)

def preprocess(text):
    # tokenize text
    tokens = word_tokenize(text.lower())
    # remove stopwords and punctuation
    tokens = [token for token in tokens if token not in stop_words]
    # lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text



def extract_weight_and_length(text):
    """
    Extracts weight and length values along with their units from text using regular expressions.
    Returns a dictionary with keys 'weight' and 'length'.
    """
    # regular expressions to detect weight and length values
    weight_pattern = r"\b(\d+(?:\.\d+)?)\s*(g|kg|pound|lb)\b"
    length_pattern = r"\b(\d+(?:\.\d+)?)\s*(m|cm|mm|kilometer|km|mile|inch|in)\b"
    size_pattren =   r'\b(\d+\s*x\s*\d+)\s*(m|cm|mm|kilometer|km|mile|inch|in)\b'


    # extract weight and length values and their units from text
    weights = re.findall(weight_pattern, text, re.IGNORECASE)
    lengths = re.findall(length_pattern, text, re.IGNORECASE)
    sizes = re.findall(size_pattren, text, re.IGNORECASE)

    # convert weight values to grams
    size_dict = {}
    for size, unit in sizes:
        size_dict['size'] = str(size) + str(unit)

    # convert length values to meters
    length_dict = {}
    for length, unit in lengths:
        
        length_dict['length'] = str(length) + str(unit)

    # combine weight and length dictionaries and return
    result = {}
    result.update(length_dict)
    result.update(size_dict)
    return result

[nltk_data] Downloading package punkt to /home/sohaib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sohaib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from tqdm import tqdm
final_output = {}

def get_tags(data, category_dict):
    response = []
    final_json = dict()
    for index, chunk in tqdm(enumerate(data)):
        final_output = dict()
        description = chunk['SKU_SHORT_DESCRIPTION'] + chunk['SKU_LONG_DESCRIPTION'] + chunk['SKU_TITLE']
        description = preprocess(description)
        final_json["description"] = description
        for category in category_dict:
            values = category_dict[category]
            words = list(set(description.split(" ")))
            words = [word.lower() for word in words]
            for item in category_dict[category]:
                item = str(item)
                item = preprocess(item)
                for word in words:

                    match_ratio = fuzz.ratio(item, word)
                    if ( match_ratio > 90):
                        if category in final_output:
                            final_output[category].append(item)
                        else:
                            final_output[category] = [item]
        
        sizes = extract_weight_and_length(description)
        final_output.update(sizes)
        final_json["tags"] = final_output
        response.append(copy.deepcopy(final_json))
    return response

resp = get_tags(data, category_dict)

# 
# print("\n\nfinal output", final_output)

1416it [11:17,  2.35it/s]

In [220]:
with open("result.json", "w") as outfile:
    json.dump(resp, outfile)